# Solving a complex task with a multi-agent hierarchy

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to there, and represent them on a map, with a color varying by cargo plane transfer time. Also represent some supercar factories with the same cargo plane transfer time.

Let's build this!

In [2]:
!pip install smolagents

      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return flight_time


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.81795798345509


In [9]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [10]:
import os
from PIL import Image
from smolagents import CodeAgent, DuckDuckGoSearchTool, OpenAIServerModel, VisitWebpageTool

model = OpenAIServerModel(model_id="gpt-4o")

We can start with creating a baseline, simple agent to give us a simple report.

In [11]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [12]:
import pandas as pd

agent = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
Here is your code snippet:
Thought: To solve this, I need to take the following steps:
1. Use a web search to find filming locations for Batman movies.
2. Use another web search to find supercar factories around the world.
3. Calculate the cargo plane travel time from each of these locations to Gotham (40.7128° N, 74.0060° W).
4. Create a pandas dataframe with the filming locations, the cargo plane travel time, and do the same for supercar 
factories.
5. Finally, return the results.

I'll start by finding the filming locations of Batman movies using a web search.
Make sure to include code with the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_code>
Make sure to provide correct code blobs.

[Step 0: Duration 3.50 seconds| Input tokens: 2,309 | Output tokens: 123]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 5: Duration 5.01 seconds| Input tokens: 28,518 | Output tokens: 1,513]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Return the DataFrames containing travel times for Batman filming locations and supercar factories              
  final_answer({                                                                                                   
      "Batman Filming Locations": batman_df,                                                                       
      "Supercar Factories": supercar_df                                                                            
  })                                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: {'Batman Filming Locations':                  Location  Travel Time (hours)
0         London, England             9.169659
1      Liverpool, England             8.808281
2       Glasgow, Scotland             8.599898
3  Chicago, Illinois, USA             2.678294
4        Watford, England             9.136400
5        Bedford, England             9.105527
6  Hertfordshire, England             9.148656
7      Knebworth, England             9.145677
8        Shotts, Scotland             8.642041
9        Northern Ireland             8.431313, 'Supercar Factories':                       Location  Travel Time 
(hours)
0              Woking, England             9.137699
1           Sheffield, England             8.952097
2            Bicester, England             9.051896
3  Sant'Agata Bolognese, Italy            10.741185
4             Maranello, Italy            10.723410
5     Fremont, California, USA             7.016536}

[Step 6: Duration 1.70 seconds| Input tokens: 36,716 | Output tokens: 1,602]

In [13]:
result

{'Batman Filming Locations':                  Location  Travel Time (hours)
 0         London, England             9.169659
 1      Liverpool, England             8.808281
 2       Glasgow, Scotland             8.599898
 3  Chicago, Illinois, USA             2.678294
 4        Watford, England             9.136400
 5        Bedford, England             9.105527
 6  Hertfordshire, England             9.148656
 7      Knebworth, England             9.145677
 8        Shotts, Scotland             8.642041
 9        Northern Ireland             8.431313,
 'Supercar Factories':                       Location  Travel Time (hours)
 0              Woking, England             9.137699
 1           Sheffield, England             8.952097
 2            Bicester, England             9.051896
 3  Sant'Agata Bolognese, Italy            10.741185
 4             Maranello, Italy            10.723410
 5     Fremont, California, USA             7.016536}

We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [14]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt-4o ────────────────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                1. **Identify Batman Filming Locations:**
   - Use the `web_search` tool to perform searches for the filming locations of various Batman movies.
   - Collect the top search results for relevant data sources.

2. **Confirm Batman Filming Locations:**
   - For each search result that lists Batman filming locations, use the `visit_webpage` tool to visit the page and
verify the numbers and locations mentioned.

3. **Gather Coordinates for Filming Locations:**
   - For each confirmed filming location, determine its geographical coordinates (latitude and longitude).

4. **Identify Supercar Factories:**
   - Use the `web_search` tool to perform searches for the locations of supercar factories around the world.
   - Collect the top search results for relevant data sources.

5. **Confirm Supercar Factory Locations:**
   - For each search result that lists supercar factory locations, use the `visit_webpage` tool to visit the page 
and verify the locations mentioned.

6. **Gather Coordinates for Supercar Factories:**
   - For each confirmed supercar factory location, determine its geographical coordinates (latitude and longitude).

7. **Calculate Cargo Plane Transfer Time for Batman Filming Locations:**
   - For each confirmed Batman filming location, use the `calculate_cargo_travel_time` tool to calculate the travel
time for a cargo plane from the location to Gotham’s coordinates (40.7128° N, 74.0060° W).

8. **Calculate Cargo Plane Transfer Time for Supercar Factories:**
   - For each confirmed supercar factory location, use the `calculate_cargo_travel_time` tool to calculate the 
travel time for a cargo plane from the location to Gotham’s coordinates (40.7128° N, 74.0060° W).

9. **Compile Data into Pandas DataFrame:**
   - Use the calculated travel times and corresponding locations to create a pandas DataFrame containing columns 
for location name, type (Batman Filming Location/Supercar Factory), coordinates, and transfer time to Gotham.

10. **Provide Final Answer:**
    - Use the `final_answer` tool to return the completed pandas DataFrame as the final answer to the task.


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_movies = [                                                                                                
      "Batman Begins filming locations",                                                                           
      "The Dark Knight filming locations",                                                                         
      "The Dark Knight Rises filming locations",                                                                   
      "Batman v Superman filming locations"                                                                        
  ]                                                                                                                
                                                                                                                   
  batman_search_results = {}                                                                                       
                                                                                                                   
  for movie in batman_movies:                                                                                      
      results = web_search(query=movie)                                                                            
      batman_search_results[movie] = results                                                                       
      print(f"Search results for {movie}:", results)                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Search results for Batman Begins filming locations: ## Search Results

[Filming Locations for Christopher Nolan's Batman Begins (2005), in 
...](https://movie-locations.com/movies/b/Batman-Begins.php)
Film locations for Christopher Nolan's Batman Begins (2005): Mentmore Towers, Mentmore, Buckinghamshire ('Wayne 
Manor'); Coalhouse Fort, East Tilbury, Essex ('Bhutanese' prison); Öræfasveit, Vatnajökuls and Svínafellsjökuls, 
Iceland ('Tibetan Himalayas'); Board of Trade Building, 141 West Jackson Boulevard, Chicago ('Wayne Enterprises' 
HQ); Garrick Theatre, Charing Cross Road ...

[Where Was Batman Begins Filmed? 2005 Movie's Real Filming 
Locations](https://thecinemaholic.com/where-was-batman-begins-filmed/)
Learn where Christopher Nolan shot his darker version of Batman in 2005. The film used a combination of real and 
fictional locations, such as London's Garrick Theatre, Chicago's Board of Trade Building, and Iceland's Vatnajökull
Glacier.

[Batman Begins (2005) - Filming & production - IMDb](https://m.imdb.com/title/tt0372784/locations/)
Batman Begins: Directed by Christopher Nolan. With Christian Bale, Michael Caine, Liam Neeson, Katie Holmes. After 
witnessing his parents' death, Bruce learns the art of fighting to confront injustice. When he returns to Gotham as
Batman, he must stop a secret society that intends to destroy the city.

[Batman Begins (2005) Film 
Locations](https://globalfilmlocations.net/2018/02/04/batman-begins-2005-film-locations/)
Batman Begins is a 2005 American superhero film based on the DC Comics character Batman. It is the first 
installment of the Batman film series and a reboot of the Batman franchise. The movie was directed by Christopher 
Nolan, who co-wrote its screenplay with David Goyer, and stars Christian Bale as Bruce Wayne/Batman. The film…

[Where was Batman Begins filmed? - 
Giggster](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Batman Begins Locations. Batman Begins was filmed in various locations across the globe, with most scenes shot in 
the United Kingdom, specifically in London, Buckinghamshire, and Essex. This gave the production a sinisterly deco 
backdrop which is further reinforced by the use of Shepperton Studios and two gigantic hangars at Cardington.

[Batman Begins - filming locations - SCEEN IT](https://www.sceen-it.com/movie/310/Batman-Begins)
See where they filmed this movie from 2005 starring Christian Bale, Michael Caine and Liam Neeson. There are more 
than 11 filming locations on our site. If you like to see Mentmore Towers in United Kingdom or any other wonderful 
filming locations from Batman Begins, start scrolling.

[Batman Begins Locations - Movies Locations - 
LatLong](https://www.latlong.net/location/batman-begins-locations-1723)
View the locations of Batman Begins on map. ... Batman Begins is an action crime drama film directed by Christopher
Nolan, written by Nolan and David S. Goyer, based on the DC Comics characters, and scored by Hans Zimmer and James 
Newton Howard. Starring an incredible cast consisting of Christian Bale, Michael Caine, Katie Holmes, Liam Neeson 
...

[Where was Batman Begins filmed? | MovieLocate](https://movielocate.com/view/batman-begins-2005/538/movie)
Batman Begins (2005) After training with his mentor, Batman begins his fight to free crime-ridden Gotham City from 
corruption. Filming Locations. Leighton Buzzard, Buckinghamshire, United Kingdom Chicago, Illinois, United States 
Bedford, Bedford, United Kingdom

[24 Filming locations for Batman Begins - Things On TV](https://www.thingson.tv/movie/batman-begins/locations)
24 Filming locations for Batman Begins. PG-13 06/10/2005. Batman Begins. Watch on... Google Play Movies; Hulu; Max 
Amazon Channel; Amazon Video; YouTube; Fandango At Home ... 53 more specific film locations in New York City, New 
York, New York, USA. 36 movies & TV shows filmed here in New York City, New York, New York, USA.

[Batman Begins | Media - Where Filmed](h

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(df)                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer:                     Location Name              Type           Coordinates  \
0                   New York City   Batman Location    (40.7128, -74.006)   
1                         Chicago   Batman Location   (41.8781, -87.6298)   
2                      Essex (UK)   Batman Location     (51.7767, 0.1521)   
3            Mentmore Towers (UK)   Batman Location    (51.8661, -0.6521)   
4   Iceland (Vatnajökull Glacier)   Batman Location   (64.4197, -16.8424)   
5                       Hong Kong   Batman Location   (22.3193, 114.1694)   
6                     London (UK)   Batman Location    (51.5074, -0.1278)   
7                      Pittsburgh   Batman Location   (40.4406, -79.9959)   
8                     Los Angeles   Batman Location  (34.0522, -118.2437)   
9                         Detroit   Batman Location   (42.3314, -83.0458)   
10                     New Mexico   Batman Location  (34.5199, -105.8701)   
11                        Ferrari  Supercar Factory    (44.5292, 10.8642)   
12                    Lamborghini  Supercar Factory    (44.6636, 11.1517)   

    Travel Time (Hours)  
0              1.000000  
1              2.678294  
2              9.182755  
3              9.101151  
4              7.527007  
5             20.001254  
6              9.169659  
7              1.743148  
8              6.772428  
9              2.134297  
10             5.213395  
11            10.723179  
12            10.741766  

[Step 7: Duration 46.28 seconds| Input tokens: 128,707 | Output tokens: 2,423]

                    Location Name              Type           Coordinates  \
0                   New York City   Batman Location    (40.7128, -74.006)   
1                         Chicago   Batman Location   (41.8781, -87.6298)   
2                      Essex (UK)   Batman Location     (51.7767, 0.1521)   
3            Mentmore Towers (UK)   Batman Location    (51.8661, -0.6521)   
4   Iceland (Vatnajökull Glacier)   Batman Location   (64.4197, -16.8424)   
5                       Hong Kong   Batman Location   (22.3193, 114.1694)   
6                     London (UK)   Batman Location    (51.5074, -0.1278)   
7                      Pittsburgh   Batman Location   (40.4406, -79.9959)   
8                     Los Angeles   Batman Location  (34.0522, -118.2437)   
9                         Detroit   Batman Location   (42.3314, -83.0458)   
10                     New Mexico   Batman Location  (34.5199, -105.8701)   
11                        Ferrari  Supercar Factory    (44.5292, 10.8642)   

In [15]:
detailed_report

,Location Name,Type,Coordinates,Travel Time (Hours)
0,New York City,Batman Location,"(40.7128, -74.006)",1.000000
1,Chicago,Batman Location,"(41.8781, -87.6298)",2.678294
2,Essex (UK),Batman Location,"(51.7767, 0.1521)",9.182755
3,Mentmore Towers (UK),Batman Location,"(51.8661, -0.6521)",9.101151
4,Iceland (Vatnajökull Glacier),Batman Location,"(64.4197, -16.8424)",7.527007
5,Hong Kong,Batman Location,"(22.3193, 114.1694)",20.001254
6,London (UK),Batman Location,"(51.5074, -0.1278)",9.169659
7,Pittsburgh,Batman Location,"(40.4406, -79.9959)",1.743148
8,Los Angeles,Batman Location,"(34.0522, -118.2437)",6.772428
9,Detroit,Batman Location,"(42.3314, -83.0458)",2.134297


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `matplotlib`, and `geopandas` + `shapely` for spatial plotting.

In [16]:
!pip install matplotlib geopandas shapely -q

In [17]:
model = OpenAIServerModel("gpt-4o")

web_agent = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=1,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it a stronger model, and add a `planning_interval` to the mix.

In [18]:
from smolagents.utils import encode_image_base64, make_image_url


model = OpenAIServerModel("o1")


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o")
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list 3 reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "But if any data was hallucinated/invented, you should refuse it. Also to pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {"type": "image_url", "image_url": {"url": make_image_url(encode_image_base64(image))}},
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=model,
    tools=[],
    managed_agents=[web_agent],
    additional_authorized_imports=["geopandas", "plotly", "shapely", "json", "pandas", "numpy"],
    planning_interval=4,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=10,
)

Let us inspect what this team looks like:

In [19]:
manager_agent.visualize()

CodeAgent | o1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gpt-4o
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a duckduckgo web search  │ query (`string`): The search      │
            │                             │ based on your query (think a      │ query to perform.                 │
            │                             │ Google search) then returns the   │                                   │
            │                             │ top search results.               │                                   │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.                         │ destination_coords (`array`):     │
            │                             │                                   │ Tuple of (latitude, longitude)    │
            │                             │                                   │ for the destination               │
            │                             │                                   │ cruising_speed_kmh (`number`):    │
            │                             │                                   │ Optional cruising speed in km/h   │
            │                             │                                   │ (defaults to 750 km/h for typical │
            │                             │                                   │ cargo planes)                     │
            │ final_answer                │ Provides a final answer to the    │ answer (`any`): The final answer  │
            │                             │ given problem.                    │ to the problem                    │
            └─────────────────────────────┴───────────────────────────────────┴───────────────────────────────────┘

In [ ]:
manager_agent.run(f"""
{task}
Represent this as spatial map of the world, with the locations represented as scatter points proportional in size to the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size="car_hours",
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=10)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)
""")

I don't know how that went in your run, but in mine, the manager agent masterfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]